In [1]:
# (\__/)    Still a little messy, but this code was written for the Mobility for All Abilities
# (•ㅅ•)     on 9/21/2019 as part of the 'Reliable Access to Subways' team organized by
# / 　 づ    TransitCenter and United Equal Access NY.

In [1]:
import PyPDF2 
from tqdm import tqdm_notebook as tqdm
import re
import pandas

In [3]:
def convert_transit_pdf_to_csv(pdf, start, end):
    
    '''
    Input is a PDF file in the local directory, the first page
    of the PDF and the last page of the PDF. The function adjusts
    the first page to account for zero-based numbering. It will
    output a csv file back into the directory.
    
    There are a few issues with the code as-written. If an escalator
    or elevator is avialable 0.00% of the time, it will add an additional
    digit or character to the availability column. There is one other
    issue I've encountered where the subway lines aren't formatted properly.
    
    The comments will occaisonaly cut off and a fix for that is the first
    priority once this code is re-visited.
    '''

    page_range = list(range(start - 1,end))

    pdfFileObj = open(pdf, 'rb') 

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    lines = []
    availability = []
    units = []
    stations = []
    conditions = []

    condition = ''

    for page in page_range:
        pageObj = pdfReader.getPage(page) 
        current_page = pageObj.extractText().split('\n')

        for i in range(len(current_page[:-2])):
            
            # removes some titles that would otherwise be caught up in 
            if not re.match(string = current_page[i], pattern = '.* THAN 85% AVAILABILITY'):
                if len(current_page[i]) > 1:
                    if len(current_page[i]) > 40:
                        condition += current_page[i]
                    if re.match(string = current_page[i][-5:], pattern = '\d\.\d{2}\%'):
                        availability.append(current_page[i][-6:])
                        lines.append(current_page[i][:-6])
                    if re.match(string = current_page[i], pattern = 'E[LS]\d{3}'):
                        units.append(current_page[i])
                        stations.append(current_page[i + 1])
                        if len(condition) > 1:
                            conditions.append(condition)
                            condition = ''
                if i == len(current_page[:-2]) - 1:
                    conditions.append(condition)
                    condition = ''
    df_stations = pd.DataFrame(
        {'units': units,
         'stations': stations,
         'lines' : lines,
         'availability' : availability,
         'condition': conditions
        })

    df_stations.to_csv(pdf + ' converted.csv')

In [419]:
convert_transit_pdf_to_csv('180521_1030_Transit.pdf', 220, 222)

ValueError: arrays must all be same length

In [399]:
df_stations

,units,stations,lines,availability,condition
0,EL120,190 St,A,27.18%,This elevator was out multiple times due to le...
1,EL249,86 St,Q,84.06%,This elevator was out of service from 11/19/18...
2,ES104,Gun Hill Rd,25,83.76%,From 10/1/18 to 12/23/18 the escalator went ou...
3,ES105,Gun Hill Rd,25,83.48%,From 10/1/18 to 12/23/18 the escalator went ou...
4,ES106,West Farms Sq-E Tremont Av,25,79.51%,From 10/1/18 to 12/23/18 the escalator went ou...
5,ES108,Intervale Av,25,69.84%,The escalator was out service from 10/17/2018 ...
6,ES114,161 St-Yankee Stadium,4,69.45%,The escalator was out of service from 10/18/20...
7,ES120,Pelham Bay Park,6,22.72%,The escalator was out of service from 09/01/20...
8,ES305,Court St,R,50.95%,This escalator was out of service from 10/6/18...
9,ES347,Broadway Junction,ACJLZ,84.90%,This escalator was out of service 13 times due...


In [420]:
len(conditions)

27

In [421]:
len(lines)

27

In [422]:
len(units)

27

In [423]:
len(stations)

27

In [424]:
len(availability)

27

In [425]:
current_page

['Borough/ Unit',
 'Location',
 '24 Hr ',
 'Availability',
 'Comments',
 '4TH QUARTER 2018 ESCALATORS WITH LESS THAN 85% AVAILABILITY',
 'ES235',
 '34 St-Herald Sq ',
 'BDFM84.34%',
 'The escalator was out of service from 12/4/18 to 12/11/18 due to worn out handrail and countershaft chains ',
 'as well as defective brake assemblies. The countershaft assembly and chain were replaced and adjusted. ',
 'The right handrail chain was adjusted. The main brakes were replaced and adjusted as well as a controller ',
 '',
 'relay; the escalator was tested and returned to service. The escalator was out multiple times due to the ',
 '',
 'activation of various safety devices. Those safety devices were tested and adjusted as needed.',
 'ES255',
 'Grand Central-42 St ',
 '4567S82.56%',
 'This escalator was out of service from 11/30/18 to 12/3/18 due to an inspection of the proper operation of ',
 'the up and down operation. The escalator was found to be operating as designed and returned to service.

In [4]:
def convert_transit_pdf_to_csv2(pdf, start, end):
    
    '''
    Input is a PDF file in the local directory, the first page
    of the PDF and the last page of the PDF. The function adjusts
    the first page to account for zero-based numbering. It will
    output a csv file back into the directory.
    
    There are a few issues with the code as-written. If an escalator
    or elevator is avialable 0.00% of the time, it will add an additional
    digit or character to the availability column. There is one other
    issue I've encountered where the subway lines aren't formatted properly.
    
    The comments will occaisonaly cut off and a fix for that is the first
    priority once this code is re-visited.
    '''

    page_range = list(range(start - 1,end))

    pdfFileObj = open(pdf, 'rb') 

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    lines = []
    availability = []
    units = []
    stations = []
    conditions = []

    condition = ''

    for page in page_range:
        pageObj = pdfReader.getPage(page) 
        current_page = pageObj.extractText().split('\n')

        for i in range(len(current_page[:-2])):
            
            # removes some titles that would otherwise be caught up in 
            if not re.match(string = current_page[i], pattern = '.* THAN 85% AVAILABILITY'):
                if len(current_page[i]) > 1:
                    if len(current_page[i]) > 40:
                        condition += current_page[i]
                    if re.match(string = current_page[i][-5:], pattern = '\d\.\d{2}\%'):
                        availability.append(current_page[i][-6:])
                        lines.append(current_page[i][:-6])
                    if re.match(string = current_page[i], pattern = 'E[LS]\d{3}'):
                        units.append(current_page[i])
                        stations.append(current_page[i + 1])
                        if len(condition) > 1:
                            conditions.append(condition)
                            condition = ''
                if i == len(current_page[:-2]) - 1:
                    conditions.append(condition)
                    condition = ''
    df_stations = pd.DataFrame(
        {'units': units,
         'stations': stations,
         'lines' : lines,
         'availability' : availability,
         'condition': conditions
        })

    df_stations.to_csv(pdf + ' converted.csv')

# A Tabula Approach

In [2]:
from tabula import read_pdf
import tabula

In [232]:
list(range(380, 384))

[380, 381, 382, 383]

In [196]:
df = tabula.read_pdf('190923_1000_Transit.pdf', pages=[380, 381, 382, 383, 384])
df.dropna(how = 'all', inplace = True)
df.reset_index(inplace = True, drop = True)

In [197]:
df.head(4)

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments
1,BROOKLYN,NaN,NaN,NaN
2,EL204,Grand Central-42 St 4567S,72.36%,This elevator was out of service on multiple d...
3,EL119,181 St A,81.16%,This elevator was out of service from 6/3/19 t...


In [224]:
unit = []
station = []
line = []
availability = []

for i in range(len(df)):
    try:
        if math.isnan(float(df.iloc[i][1])) is True:
            pass
    except:
        if df.iloc[i][1] != 'Location':
            unit.append(df.iloc[i][0])
            station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
            line.append(df.iloc[i][1].split(' ')[-1])
    if df.iloc[i][1] != 'Location':
        if df.iloc[i][2] != 'Location':
            if df.iloc[i].isna().sum() == 0:
                availability.append(df.iloc[i][2])
            if df.iloc[i].isna().sum() == 1:
                availability.append(df.iloc[i][3].split(' ')[0])
            
comment = ''
comments = []

for i in range(len(df_revision)):
    
    if df_comment['starts percent'][i] == True:
        if len(comment) > 0:
            comments.append(comment)
            comment = ''
        comment += ' '.join(df_revision['Unnamed: 3'][i].split(' ')[1:])
        
    if df_comment['continued comment'][i] == True and df_null['Unnamed: 2'][i] == True:
        comment += df_revision['Unnamed: 3'][i]
        
    if df_comment['full comment'][i] == True:
        if len(comment) > 0:
            comments.append(comment)
            comment = ''
        comments.append(df_revision['Unnamed: 3'][i])
            
df_stations = pd.DataFrame(
    {'unit': unit,
     'station': station,
     'line' : line,
     'availability' : availability,
     'comments' : comments
    })

In [225]:
df_stations

,unit,station,line,availability,comments
0,EL204,Grand Central-42 St,4567S,72.36%,This elevator was out of service on multiple d...
1,EL119,181 St,A,81.16%,This elevator was out of service from 6/3/19 t...
2,EL224,8 Av,L,83.42%,This elevator was out of service on multiple o...
3,ES104,Gun Hill Rd,25,74.40%,This escalator had multiple outages for lower ...
4,ES105,Gun Hill Rd,25,79.55%,This escalator was out of service multiple tim...
5,ES106,West Farms Sq-E Tremont Av,25,68.09%,This escalator was out of service multiple tim...
6,ES108,Intervale Av,25,34.94%,This escalator was out of service from 4/23/19...
7,ES112,Norwood-205 St,D,66.57%,This escalator was out of service from 4/19/19...
8,ES113,161 St-Yankee Stadium,4,73.03%,This escalator had repeated outages for combst...
9,ES114,161 St-Yankee Stadium,4,48.10%,This escalator was out of service from 4/11/19...


In [161]:

type(df.iloc[1][1])

float

In [244]:
df

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
2,BROOKLYN,NaN,NaN,NaN
3,EL204,Grand Central-42 St 4567S,72.36%,This elevator was out of service on multiple d...
4,EL119,181 St A,81.16%,This elevator was out of service from 6/3/19 t...
5,EL224,8 Av L,83.42%,This elevator was out of service on multiple o...
6,NaN,NaN,NaN,2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
7,Borough/ Unit,NaN,Location,24 Hr Comments
8,NaN,NaN,NaN,Availability
9,BRONX,NaN,NaN,NaN
10,ES104,Gun Hill Rd 25,NaN,74.40% This escalator had multiple outages for...
11,NaN,NaN,NaN,device activations. The escalator was out of s...


In [14]:
import math

In [214]:
math.isnan(float(df.iloc[0][1]))

True

In [334]:
for i in range(len(df)):
    try:
        if math.isnan(float(df.iloc[i][1])) is True:
            pass
    except:
        if re.match(string = df.iloc[i][-1].split(' ')[0], pattern = '\d+\.\d{2}\%'):
            print(' '.join(df.iloc[i][-1].split(' ')[1:]))

This escalator had multiple outages for lower combstop safety device activations and step-up thrust safety
This escalator was out of service multiple times for various activations of the escalator safety devices. The
This escalator was out of service multiple times due to vandalism. The emergency stop buttons were being
This escalator was out of service from 4/23/19 to 6/14/19 due to a step wreck. Various step chains and step
This escalator was out of service from 4/19/19 to 5/17/19 due to a safety check and related work. The comb
This escalator had repeated outages for combstop and comb impact safety activations. The escalator was
This escalator was out of service from 4/11/19 to 5/17/19 due to a safety check and related repair work.
This escalator was out of service from 4/16/19 to 4/23/19 for a safety check and related work. The skirts,
This escalator had various multiple outages for phase reversal, combstop safety devices activating and
This escalator was out of service from 4/13/1

In [356]:
df[df.columns[-1]]

0                                               Comments
1                                                    NaN
2      This elevator was out of service on multiple d...
3      This elevator was out of service from 6/3/19 t...
4      This elevator was out of service on multiple o...
5      2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6                                         24 Hr Comments
7                                           Availability
8                                                    NaN
9      74.40% This escalator had multiple outages for...
10     device activations. The escalator was out of s...
11     correct repeated outages for the lower combsto...
12     were adjusted; the escalator was tested and re...
13     79.55% This escalator was out of service multi...
14     upper and lower emergency stop buttons were ac...
15     combstop safety devices were activated. Each t...
16     service. Further work is required to inspect, ...
17     combstop safety devices 

In [384]:
comment = ''

In [488]:
comments = []
comment = ''

raw_comments = df[df.columns[-1]]

for i in range(len(raw_comments)):
    try:
        if math.isnan(float(raw_comments[i])) is True:
            pass
    except:
        if len(raw_comments[i]) > 12:
            if re.match(string = raw_comments[i], pattern = '\d+\.\d{2}\%'):
                comments.append(comment)
                comment = ''
                comment += ' '.join(raw_comments[i].split(' ')[1:])
            if re.match(string = raw_comments[i], pattern = '^[Th]'):
                comments.append(raw_comments[i])
            if re.match(string = raw_comments[i], pattern = '^[a-zA-Z]'):
                comment += raw_comments[i]
                
comments = list(filter(None, comments))
                
for j in range(len(comments)):
    comments[j] = comments[j].replace("\r", " ")

In [491]:
len(comments)

64

In [490]:
comments

['This elevator was out of service on multiple days in April due to planned maintenance repairs. The elevator was out of service from 5/6/19 to 5/8/19 due to the hydraulic jack leaking. The hydraulic jack seals were replaced and the piston was filed; the elevator was tested and returned to service. The elevator was out of service from 5/13/19 to 5/15/19 due to an entrapment investigation. Repairs and adjustments were made to the car and hoistway doors and interlocks, rail guide clamps in the shaftway were secured and the elevator cab was squared and secured; the elevator was tested and returned to service. The elevator is scheduled for replacement in the 2015-2019 Capital Replacement Program; the replacement work is expected to start later this year.',
 'This elevator was out of service from 6/3/19 to 6/13/19 due to defective elevator brakes. The elevator brakes were repaired; the elevator was tested and returned to service. he elevator is scheduled for replacement in the 2015-2019 Cap

In [470]:
len(set(comments)) - len(comments)

0

In [472]:
len(comments)

63

In [463]:
import pandas as pd

In [464]:
pd.options.display.max_rows = 4000

In [465]:
raw_comments

0                                               Comments
1                                                    NaN
2      This elevator was out of service on multiple d...
3      This elevator was out of service from 6/3/19 t...
4      This elevator was out of service on multiple o...
5      2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6                                         24 Hr Comments
7                                           Availability
8                                                    NaN
9      74.40% This escalator had multiple outages for...
10     device activations. The escalator was out of s...
11     correct repeated outages for the lower combsto...
12     were adjusted; the escalator was tested and re...
13     79.55% This escalator was out of service multi...
14     upper and lower emergency stop buttons were ac...
15     combstop safety devices were activated. Each t...
16     service. Further work is required to inspect, ...
17     combstop safety devices 

In [411]:
q = 0
for i in range(len(comments)):
    if len(comments[i]) == 0:
        q += 1
q

1

In [360]:
print(re.match(string = raw_comments[10], pattern = '^[a-z]'))

<re.Match object; span=(0, 1), match='d'>


'device'

'nan'

In [131]:
df.iloc[i][1].split(' ')[-1]

AttributeError: 'float' object has no attribute 'split'

no


4

In [4]:
comments = []
comment = ''

raw_comments = df[df.columns[-1]]

In [64]:
raw_comments

0                                               Comments
1                                                    NaN
2      This elevator was out of service on multiple d...
3      This elevator was out of service from 6/3/19 t...
4      This elevator was out of service on multiple o...
5      2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6                                         24 Hr Comments
7                                           Availability
8                                                    NaN
9      74.40% This escalator had multiple outages for...
10     device activations. The escalator was out of s...
11     correct repeated outages for the lower combsto...
12     were adjusted; the escalator was tested and re...
13     79.55% This escalator was out of service multi...
14     upper and lower emergency stop buttons were ac...
15     combstop safety devices were activated. Each t...
16     service. Further work is required to inspect, ...
17     combstop safety devices 

In [86]:
thread = 0
for i in range(len(raw_comments)):
    try:
        if re.match(string = raw_comments[i][:3], pattern = '^Th[ei].*$'):
            print(raw_comments[i][:3])
    except:
        pass

Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
Thi
The
The
The
The
Thi
The
Thi
Thi
The
Thi
The
The


In [52]:
raw_comments[i]

'The escalator was out of service from 4/22/19 to 7/2/19 due to extensive escalator repairs. The gearcase\rwere removed and replaced; the escalator was tested and returned to service.'

In [100]:
if re.match(string = raw_comments[2], pattern = '^(Th).*$'):
    result = raw_comments[2]

In [101]:
result

'This elevator was out of service on multiple days in April due to planned maintenance repairs. The\relevator was out of service from 5/6/19 to 5/8/19 due to the hydraulic jack leaking. The hydraulic\rjack seals were replaced and the piston was filed; the elevator was tested and returned to service.\rThe elevator was out of service from 5/13/19 to 5/15/19 due to an entrapment investigation. Repairs\rand adjustments were made to the car and hoistway doors and interlocks, rail guide clamps in the\rshaftway were secured and the elevator cab was squared and secured; the elevator was tested and\rreturned to service. The elevator is scheduled for replacement in the 2015-2019 Capital\rReplacement Program; the replacement work is expected to start later this year.'

In [95]:
raw_comments[2]

'This elevator was out of service on multiple days in April due to planned maintenance repairs. The\relevator was out of service from 5/6/19 to 5/8/19 due to the hydraulic jack leaking. The hydraulic\rjack seals were replaced and the piston was filed; the elevator was tested and returned to service.\rThe elevator was out of service from 5/13/19 to 5/15/19 due to an entrapment investigation. Repairs\rand adjustments were made to the car and hoistway doors and interlocks, rail guide clamps in the\rshaftway were secured and the elevator cab was squared and secured; the elevator was tested and\rreturned to service. The elevator is scheduled for replacement in the 2015-2019 Capital\rReplacement Program; the replacement work is expected to start later this year.'

In [102]:
df

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments
1,BROOKLYN,NaN,NaN,NaN
2,EL204,Grand Central-42 St 4567S,72.36%,This elevator was out of service on multiple d...
3,EL119,181 St A,81.16%,This elevator was out of service from 6/3/19 t...
4,EL224,8 Av L,83.42%,This elevator was out of service on multiple o...
5,NaN,NaN,NaN,2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6,Borough/ Unit,NaN,Location,24 Hr Comments
7,NaN,NaN,NaN,Availability
8,BRONX,NaN,NaN,NaN
9,ES104,Gun Hill Rd 25,NaN,74.40% This escalator had multiple outages for...


In [110]:
df['Unnamed: 3'].str.contains('^(Th).*$')

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


0      False
1        NaN
2       True
3       True
4       True
5      False
6      False
7      False
8        NaN
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
111    False
112     True
113     True
114     True
115     True
116     True
117     True
118     True
119     True
120     True
121     True
122     True
123     True
124     True
125     True
126     True
127     True
128     True
129     True
130      NaN
131     True
132     True
133      NaN
134    False
135     True
136     True
137     True
138     True
139     True
140     True
Name: Unnamed: 3, Length: 141, dtype: object

In [113]:
df['Unnamed: 3'].str.contains('\S+\%')

0      False
1        NaN
2      False
3      False
4      False
5       True
6      False
7      False
8        NaN
9       True
10     False
11     False
12     False
13      True
14     False
15     False
16     False
17     False
18      True
19     False
20     False
21     False
22     False
23     False
24      True
25     False
26     False
27     False
28      True
29     False
       ...  
111    False
112    False
113    False
114    False
115    False
116    False
117    False
118    False
119    False
120    False
121    False
122    False
123    False
124    False
125    False
126    False
127    False
128    False
129    False
130      NaN
131    False
132    False
133      NaN
134    False
135    False
136    False
137    False
138    False
139    False
140    False
Name: Unnamed: 3, Length: 141, dtype: object

In [115]:
df['Unnamed: 3'].str.contains('[a-zA-SU-Z]')

0      True
1       NaN
2      True
3      True
4      True
5      True
6      True
7      True
8       NaN
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
111    True
112    True
113    True
114    True
115    True
116    True
117    True
118    True
119    True
120    True
121    True
122    True
123    True
124    True
125    True
126    True
127    True
128    True
129    True
130     NaN
131    True
132    True
133     NaN
134    True
135    True
136    True
137    True
138    True
139    True
140    True
Name: Unnamed: 3, Length: 141, dtype: object

In [117]:
import pandas as pd

In [140]:
df_comment = pd.DataFrame(
    {'full comment': df['Unnamed: 3'].str.contains('^(Th).*$'),
     'starts percent': df['Unnamed: 3'].str.contains('\d{1,2}\.\d{2}\%'),
     'continued comment': df['Unnamed: 3'].str.contains('^[a-zA-SU-Z]*[\s]'),
     'all caps': df['Unnamed: 3'].str.contains('^[^a-z]*$'),
     'no spaces': df['Unnamed: 3'].str.contains('^[^\s]*$'),
    })

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [141]:
df_comment

,full comment,starts percent,continued comment,all caps,no spaces
0,False,False,False,False,True
1,NaN,NaN,NaN,NaN,NaN
2,True,False,False,False,False
3,True,False,False,False,False
4,True,False,False,False,False
5,False,False,False,True,False
6,False,False,False,False,False
7,False,False,False,False,True
8,NaN,NaN,NaN,NaN,NaN
9,False,True,False,False,False


In [126]:
df

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments
1,BROOKLYN,NaN,NaN,NaN
2,EL204,Grand Central-42 St 4567S,72.36%,This elevator was out of service on multiple d...
3,EL119,181 St A,81.16%,This elevator was out of service from 6/3/19 t...
4,EL224,8 Av L,83.42%,This elevator was out of service on multiple o...
5,NaN,NaN,NaN,2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6,Borough/ Unit,NaN,Location,24 Hr Comments
7,NaN,NaN,NaN,Availability
8,BRONX,NaN,NaN,NaN
9,ES104,Gun Hill Rd 25,NaN,74.40% This escalator had multiple outages for...


In [138]:
df_null = df.isna()

In [139]:
df_null

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,False,False,False,False
1,False,True,True,True
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,True,True,True,False
6,False,True,False,False
7,True,True,True,False
8,False,True,True,True
9,False,False,True,False


In [205]:
df_revision = df.copy()

In [199]:
df.columns

Index(['2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')

In [230]:
# oh god this is so important
df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

In [217]:
comment = ''
comments = []

for i in range(len(df_revision)):
    
    if df_comment['starts percent'][i] == True:
        if len(comment) > 0:
            comments.append(comment)
            comment = ''
        comment += df_revision['Unnamed: 3'][i]
        
    if df_comment['continued comment'][i] == True and df_null['Unnamed: 2'][i] == True:
        comment += df_revision['Unnamed: 3'][i]
        
    if df_comment['full comment'][i] == True:
        if len(comment) > 0:
            comments.append(comment)
            comment = ''
        comments.append(df_revision['Unnamed: 3'][i])

In [219]:
len(comments)

62

In [226]:
unit = []
station = []
line = []
availability = []
comment = ''
comments = []

for i in range(len(df)):
    try:
        if math.isnan(float(df.iloc[i][1])) is True:
            pass
    except:
        if df.iloc[i][1] != 'Location':
            unit.append(df.iloc[i][0])
            station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
            line.append(df.iloc[i][1].split(' ')[-1])
    if df.iloc[i][1] != 'Location':
        if df.iloc[i][2] != 'Location':
            if df.iloc[i].isna().sum() == 0:
                availability.append(df.iloc[i][2])
            if df.iloc[i].isna().sum() == 1:
                availability.append(df.iloc[i][3].split(' ')[0])
    if df_comment['starts percent'][i] == True:
        if len(comment) > 0:
            comments.append(comment)
            comment = ''
        comment += ' '.join(df_revision['Unnamed: 3'][i].split(' ')[1:])
        
    if df_comment['continued comment'][i] == True and df_null['Unnamed: 2'][i] == True:
        comment += df_revision['Unnamed: 3'][i]
        
    if df_comment['full comment'][i] == True:
        if len(comment) > 0:
            comments.append(comment)
            comment = ''
        comments.append(df_revision['Unnamed: 3'][i])
            
df_stations = pd.DataFrame(
    {'unit': unit,
     'station': station,
     'line' : line,
     'availability' : availability,
     'comments' : comments
    })

In [227]:
df_stations

,unit,station,line,availability,comments
0,EL204,Grand Central-42 St,4567S,72.36%,This elevator was out of service on multiple d...
1,EL119,181 St,A,81.16%,This elevator was out of service from 6/3/19 t...
2,EL224,8 Av,L,83.42%,This elevator was out of service on multiple o...
3,ES104,Gun Hill Rd,25,74.40%,This escalator had multiple outages for lower ...
4,ES105,Gun Hill Rd,25,79.55%,This escalator was out of service multiple tim...
5,ES106,West Farms Sq-E Tremont Av,25,68.09%,This escalator was out of service multiple tim...
6,ES108,Intervale Av,25,34.94%,This escalator was out of service from 4/23/19...
7,ES112,Norwood-205 St,D,66.57%,This escalator was out of service from 4/19/19...
8,ES113,161 St-Yankee Stadium,4,73.03%,This escalator had repeated outages for combst...
9,ES114,161 St-Yankee Stadium,4,48.10%,This escalator was out of service from 4/11/19...


In [233]:
start = 235
end = 238
pdf = '190520_1030_Transit.pdf'

In [236]:
list(range(start, end))

[235, 236, 237]

In [355]:
def pdf_transform(start, end, pdf):

    df = tabula.read_pdf(pdf, pages=start)

    for i in range(len(df.columns)):
        df.rename(columns = {df.columns[i]:str(i)}, inplace = True) 

    for i in range(start + 1, end):
        df2 = tabula.read_pdf(pdf, pages=[i])
        for i in range(len(df2.columns)):
            df2.rename(columns = {df2.columns[i]:str(i)}, inplace = True) 

        df = pd.concat([df, df2], axis=0).reset_index(drop = True)

    df.dropna(how = 'all', inplace = True)
    df.reset_index(inplace = True, drop = True)

    unit = []
    station = []
    line = []
    availability = []
    comment = ''
    comments = []

    if len(df.columns) > 4:
        df = df[list(df.columns[:-1])]

    df_revision = df.copy()
    df_null = df.isna()

    df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

    df_comment = pd.DataFrame(
        {'full comment': df['3'].str.contains('^(Th).*$'),
         'starts percent': df['3'].str.contains('\d{1,2}\.\d{2}\%'),
         'continued comment': df['3'].str.contains('^[a-zA-SU-Z]*[\s]'),
         'all caps': df['3'].str.contains('^[^a-z]*$'),
         'no spaces': df['3'].str.contains('^[^\s]*$'),
        })

    for i in range(len(df)):
        try:
            if math.isnan(float(df.iloc[i][1])) is True:
                pass
        except:
            if df.iloc[i][1] != 'Location':
                if re.match(string = df.iloc[i][0], pattern = 'E|[SL]\d{3}'):
                    unit.append(df.iloc[i][0])
                station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
                line.append(df.iloc[i][1].split(' ')[-1])
            if df.iloc[i].isna().sum() == 0:
                if re.match(string = df.iloc[i][2], pattern = '\d{1,2}\.\d{2}\%'):
                    availability.append(df.iloc[i][2])
            if df.iloc[i].isna().sum() == 1:
                if re.match(string = df.iloc[i][3].split(' ')[0], pattern = '\d{1,2}\.\d{2}\%'):
                    availability.append(df.iloc[i][3].split(' ')[0])
            if df_comment['starts percent'][i] == True:
                if len(comment) > 0:
                    comments.append(comment)
                    comment = ''
                comment += ' '.join(df_revision['3'][i].split(' ')[1:])

            if df_comment['continued comment'][i] == True and df_null['2'][i] == True:
                comment += df_revision['3'][i]

            if df_comment['full comment'][i] == True:
                if len(comment) > 0:
                    comments.append(comment)
                    comment = ''
                comments.append(df_revision['3'][i])

    df_stations = pd.DataFrame(
        {'unit': unit,
         'station': station,
         'line' : line,
         'availability' : availability,
         'comments' : comments
        })

    return df_stations

In [356]:
pdf_transform(235, 238, '190520_1030_Transit.pdf')

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,unit,station,line,availability,comments
0,EL340,Franklin Av,CS,78.96%,This elevator had 2 incidents and it was out o...
1,EL312,Clark St,23,81.47%,This elevator was out of service 8 times due t...
2,ES104,Gun Hill Rd,25,82.89%,This escalator was out of service from 1/7/19 ...
3,ES106,West Farms Sq-E Tremont Av,25,77.55%,This escalator was out of service from 12/28/1...
4,ES108,Intervale Av,25,78.18%,This escalator was out of service from 12/29/1...
5,ES111,Parkchester,6,79.77%,This escalator was out of service from 1/21/19...
6,ES113,161 St-Yankee Stadium,4,82.23%,This escalator was out of service from 1/10/09...
7,ES114,161 St-Yankee Stadium,4,60.20%,This escalator was out of service from 1/20/19...
8,ES120,Pelham Bay Park,6,68.77%,This escalator was out of service from 1/25/19...
9,ES122,Pelham Pkwy,25,82.74%,This escalator was out of service from 1/12/19...


In [352]:
availability

['78.96%',
 '81.47%',
 '82.89%',
 '77.55%',
 '78.18%',
 '79.77%',
 '82.23%',
 '60.20%',
 '68.77%',
 '82.74%',
 '76.73%',
 '77.44%',
 '83.99%',
 '75.73%',
 '62.65%',
 '77.09%',
 '81.62%',
 '80.96%',
 '45.58%',
 '64.43%',
 '76.29%',
 '73.31%',
 '74.45%',
 '79.43%',
 '83.90%',
 '82.19%',
 '76.40%',
 '78.74%',
 '81.31%',
 '82.19%',
 '82.74%',
 '84.42%',
 '81.43%',
 '47.47%',
 '81.63%',
 '80.09%',
 '82.89%',
 '70.37%',
 '83.83%',
 '79.92%',
 '63.20%']

In [324]:
df

,0,1,2,3,4
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments,NaN
1,BROOKLYN,NaN,NaN,NaN,NaN
2,EL340,Franklin Av CS,78.96%,This elevator had 2 incidents and it was out o...,NaN
3,EL312,Clark St 23,81.47%,This elevator was out of service 8 times due t...,NaN
4,Borough/ Unit,NaN,Location,24 Hr Comments,NaN
5,NaN,NaN,NaN,Availability,NaN
6,BRONX,NaN,NaN,NaN,NaN
7,ES104,Gun Hill Rd 25,NaN,82.89% This escalator was out of service from ...,NaN
8,NaN,NaN,NaN,repair. The escalator was out of service from ...,NaN
9,NaN,NaN,NaN,"service numerous times for up thrust track, co...",NaN


In [317]:
df

,0,1,2,3,4
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments,NaN
1,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,NaN,NaN,NaN,NaN
3,EL340,Franklin Av CS,78.96%,This elevator had 2 incidents and it was out o...,NaN
4,EL312,Clark St 23,81.47%,This elevator was out of service 8 times due t...,NaN
5,Borough/ Unit,NaN,Location,24 Hr Comments,NaN
6,NaN,NaN,NaN,Availability,NaN
7,BRONX,NaN,NaN,NaN,NaN
8,ES104,Gun Hill Rd 25,NaN,82.89% This escalator was out of service from ...,NaN
9,NaN,NaN,NaN,repair. The escalator was out of service from ...,NaN


In [273]:
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3, 9], ['d', 4, 9]], columns=['letter', 'number', 'whatever'])

In [358]:
start = 328
end = 330
pdf = '190225_1030_transit-bus.pdf'

df = tabula.read_pdf(pdf, pages=start)

for i in range(len(df.columns)):
    df.rename(columns = {df.columns[i]:str(i)}, inplace = True) 

for i in range(start + 1, end):
    df2 = tabula.read_pdf(pdf, pages=[i])
    for i in range(len(df2.columns)):
        df2.rename(columns = {df2.columns[i]:str(i)}, inplace = True) 

    df = pd.concat([df, df2], axis=0).reset_index(drop = True)

df.dropna(how = 'all', inplace = True)
df.reset_index(inplace = True, drop = True)

unit = []
station = []
line = []
availability = []
comment = ''
comments = []

if len(df.columns) > 4:
    df = df[list(df.columns[:-1])]

df_revision = df.copy()
df_null = df.isna()

df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

df_comment = pd.DataFrame(
    {'full comment': df['3'].str.contains('^(Th).*$'),
     'starts percent': df['3'].str.contains('\d{1,2}\.\d{2}\%'),
     'continued comment': df['3'].str.contains('^[a-zA-SU-Z]*[\s]'),
     'all caps': df['3'].str.contains('^[^a-z]*$'),
     'no spaces': df['3'].str.contains('^[^\s]*$'),
    })

for i in range(len(df)):
    try:
        if math.isnan(float(df.iloc[i][1])) is True:
            pass
    except:
        if df.iloc[i][1] != 'Location':
            if re.match(string = df.iloc[i][0], pattern = 'E|[SL]\d{3}'):
                unit.append(df.iloc[i][0])
            station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
            line.append(df.iloc[i][1].split(' ')[-1])
        if df.iloc[i].isna().sum() == 0:
            if re.match(string = df.iloc[i][2], pattern = '\d{1,2}\.\d{2}\%'):
                availability.append(df.iloc[i][2])
        if df.iloc[i].isna().sum() == 1:
            if re.match(string = df.iloc[i][3].split(' ')[0], pattern = '\d{1,2}\.\d{2}\%'):
                availability.append(df.iloc[i][3].split(' ')[0])
        if df_comment['starts percent'][i] == True:
            if len(comment) > 0:
                comments.append(comment)
                comment = ''
            comment += ' '.join(df_revision['3'][i].split(' ')[1:])

        if df_comment['continued comment'][i] == True and df_null['2'][i] == True:
            comment += df_revision['3'][i]

        if df_comment['full comment'][i] == True:
            if len(comment) > 0:
                comments.append(comment)
                comment = ''
            comments.append(df_revision['3'][i])

df_stations = pd.DataFrame(
    {'unit': unit,
     'station': station,
     'line' : line,
     'availability' : availability,
#      'comments' : comments
    })

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [359]:
df_stations

,unit,station,line,availability
0,EL120,190 St,A,27.18%
1,EL249,86 St,Q,84.06%
2,ES104,Gun Hill Rd,25,83.76%
3,ES105,Gun Hill Rd,25,83.48%
4,ES106,West Farms Sq-E Tremont Av,25,79.51%
5,ES108,Intervale Av,25,69.84%
6,ES114,161 St-Yankee Stadium,4,69.45%
7,ES120,Pelham Bay Park,6,22.72%
8,ES305,Court St,R,50.95%
9,ES347,Broadway Junction,ACJLZ,84.90%


In [365]:
comments[-1]

'The escalator went out of service multiple times due to the activation of the comb stop safety device. The'

In [362]:
df

,0,1,2,3
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments
1,MANHATTAN,NaN,NaN,NaN
2,EL120,190 St A,27.18%,This elevator was out multiple times due to le...
3,EL249,86 St Q,84.06%,This elevator was out of service from 11/19/18...
4,Borough/ Unit,NaN,Location,24 Hr Comments
5,NaN,NaN,NaN,Availability
6,BRONX,NaN,NaN,NaN
7,ES104,Gun Hill Rd 25,NaN,83.76% From 10/1/18 to 12/23/18 the escalator ...
8,NaN,NaN,NaN,activation due to vandalism. From 10/1/18 to 1...
9,NaN,NaN,NaN,to the comb stop safety device being too sensi...


In [372]:
start = 213
end = 214
pdf = '181113_1030_Transit.pdf'

In [373]:
df = tabula.read_pdf(pdf, pages=start)


In [374]:
df.head()

,Unnamed: 0,Unnamed: 1,3RD QUARTER 2018 ELEVATORS WITH LESS THAN 85% AVAILABILITY
0,Borough/ Unit,NaN,Location 24 Hr Comments
1,NaN,NaN,Availability
2,BROOKLYN,NaN,NaN
3,EL310,Clark St 23,73.82% This elevator was out of service from 7...
4,NaN,NaN,"bearings were replaced, the brakes were also r..."


In [375]:
df = tabula.read_pdf(pdf, pages = start + 1)

In [376]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,3RD QUARTER 2018 ESCALATORS WITH LESS THAN 85% AVAILABILITY
0,Borough/ Unit,NaN,Location,24 Hr Comments
1,NaN,NaN,NaN,Availability
2,BRONX,NaN,NaN,NaN
3,ES104,Gun Hill Rd 25,NaN,83.92% This escalator was removed from service...
4,NaN,NaN,NaN,escalator was tested and returned to service o...


In [370]:

for i in range(len(df.columns)):
    df.rename(columns = {df.columns[i]:str(i)}, inplace = True) 

for i in range(start + 1, end):
    df2 = tabula.read_pdf(pdf, pages=[i])
    for i in range(len(df2.columns)):
        df2.rename(columns = {df2.columns[i]:str(i)}, inplace = True) 

    df = pd.concat([df, df2], axis=0).reset_index(drop = True)

df.dropna(how = 'all', inplace = True)
df.reset_index(inplace = True, drop = True)

In [371]:
df.head()

,0,1,2
0,Borough/ Unit,NaN,Location 24 Hr Comments
1,NaN,NaN,Availability
2,BROOKLYN,NaN,NaN
3,EL310,Clark St 23,73.82% This elevator was out of service from 7...
4,NaN,NaN,"bearings were replaced, the brakes were also r..."


In [ ]:
unit = []
station = []
line = []
availability = []
comment = ''
comments = []

if len(df.columns) > 4:
    df = df[list(df.columns[:-1])]

df_revision = df.copy()
df_null = df.isna()

df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

df_comment = pd.DataFrame(
    {'full comment': df['3'].str.contains('^(Th).*$'),
     'starts percent': df['3'].str.contains('\d{1,2}\.\d{2}\%'),
     'continued comment': df['3'].str.contains('^[a-zA-SU-Z]*[\s]'),
     'all caps': df['3'].str.contains('^[^a-z]*$'),
     'no spaces': df['3'].str.contains('^[^\s]*$'),
    })

for i in range(len(df)):
    try:

        if math.isnan(float(df.iloc[i][1])) is True:
            pass

    except:

        if df.iloc[i][1] != 'Location':
            if re.match(string = df.iloc[i][0], pattern = 'E|[SL]\d{3}'):
                unit.append(df.iloc[i][0])
            station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
            line.append(df.iloc[i][1].split(' ')[-1])

        if df.iloc[i].isna().sum() == 0:
            if re.match(string = df.iloc[i][2], pattern = '\d{1,2}\.\d{2}\%'):
                availability.append(df.iloc[i][2])

        if df.iloc[i].isna().sum() == 1:
            if re.match(string = df.iloc[i][3].split(' ')[0], pattern = '\d{1,2}\.\d{2}\%'):
                availability.append(df.iloc[i][3].split(' ')[0])

        if df_comment['starts percent'][i] == True:
            if len(comment) > 0:
                comments.append(comment)
                comment = ''
            comment += ' '.join(df_revision['3'][i].split(' ')[1:])

        if df_comment['full comment'][i] == True and df_null['2'][i] == True:
            comment += df_revision['3'][i]

        if df_comment['full comment'][i] == True and df_null['2'][i] == False:
            if len(comment) > 0:
                comments.append(comment)
                comment = ''
            comments.append(df_revision['3'][i])

        if df_comment['continued comment'][i] == True:
            comment += df_revision['3'][i]

if len(comment) > 0:
    comments.append(comment)

df_stations = pd.DataFrame(
    {'unit': unit,
     'station': station,
     'line' : line,
     'availability' : availability,
     'comments' : comments
    })